In [1]:
import csv
import codecs
import unicodecsv
import requests
from bs4 import BeautifulSoup
import json

all_region_file_name = "./2017Data/Week6Stats.csv"
output_player_file_name = "./2017Data/Week6Stats_Players.csv"
output_champion_file_name = "./2017Data/Week6Stats_Champions.csv"

In [2]:
all_region_define_row = None
all_region_table = []

PLAYERID_COLUMN = 8
DPM_COLUMN = 59 + 2 #每分钟对英雄伤害
GPM_COLUMN = 74 #每分钟金钱
DAMAGE_TO_CHAMPIONS_COLUMN = 58 + 2 #对英雄总伤害
KILL_COLUMN = 19 + 2
DEATH_COLUMN = 20 + 2
ASSIST_COLUMN = 21 + 2
LEAGUE_COLUMN = 2
SPLIT_COLUMN = 3
WEEK_COLUMN = 5 #LPL和其余赛区的格式不同：LPL直接是周数，其他赛区是week.day
GAME_COLUMN = 6
URL_COLUMN = 1 #LPL赛区没有这栏信息
SIDE_COLUMN = 9
POSITION_COLUMN = 10
PLAYER_COLUMN = 11
TEAM_COLUMN = 12
CHAMPION_COLUMN = 13
BAN1_COLUMN = 14
BAN2_COLUMN = 15
BAN3_COLUMN = 16
BAN4_COLUMN = 17
BAN5_COLUMN = 18

GAMELENGTH_COLUMN = 17 + 2
RESULT_COLUMN = 18 + 2#0 for lose, 1 for win
TOTAL_GOLD_COLUMN = 73

CUSTOM_WEEK_COLUMN = 95
CUSTOM_DAMAGE_TAKEN_COLUMN = 96

with open(all_region_file_name, 'rb') as csvfile:
    result = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row_idx, row in enumerate(result):
        if row_idx == 0:
            all_region_define_row = row
        else:
            all_region_table.append(row)

In [3]:
def AddToDict(a_dict, a_key, a_value):
    if a_key in a_dict.keys():
        a_dict[a_key] += a_value
    else:
        a_dict[a_key] = a_value

In [4]:
#选手榜
player_dpm_sum_dict = {}
player_gpm_sum_dict = {}
player_num_dict = {}
player_ka_damage_sum_dict = {}
player_d_damage_taken_sum_dict = {}
player_damage_to_gold_sum_dict = {}
player_damage_to_gold_share_sum_dict = {}
player_damage_share_sum_dict = {}
player_damage_taken_share_sum_dict = {}
player_kill_sum_dict = {}
player_death_sum_dict = {}
player_assist_sum_dict = {}
player_win_sum_dict = {}
player_champs_dict = {}
player_league_dict = {}
player_position_dict = {}

temp_total_damage_taken_team = 0
temp_total_damage_team = 0
temp_total_gold_team = 0

for a_idx, a_row in enumerate(all_region_table):
    a_player = a_row[PLAYER_COLUMN]
    a_team = a_row[TEAM_COLUMN]
    print a_row[DPM_COLUMN]
    a_total_damage = float(a_row[DAMAGE_TO_CHAMPIONS_COLUMN])
    a_game_length = float(a_row[GAMELENGTH_COLUMN])
    a_dpm = a_total_damage / a_game_length
    a_total_gold = float(a_row[TOTAL_GOLD_COLUMN])
    a_gpm = (a_total_gold - 500) / a_game_length #500 is the starting gold
    
    AddToDict(player_dpm_sum_dict, a_player, a_dpm)
    AddToDict(player_gpm_sum_dict, a_player, a_gpm)
    AddToDict(player_num_dict, a_player, 1)
    
    
    a_kill = int(a_row[KILL_COLUMN])
    a_death = int(a_row[DEATH_COLUMN])
    a_assist = int(a_row[ASSIST_COLUMN])

    a_average_ka_damage = 0
    if a_kill + a_assist == 0:
        a_average_ka_damage = float(a_total_damage)
    else:
        a_average_ka_damage = float(a_total_damage) / float(int(a_kill) + int(a_assist))
    AddToDict(player_ka_damage_sum_dict, a_player, a_average_ka_damage)
    
    a_total_damage_taken = float(a_row[CUSTOM_DAMAGE_TAKEN_COLUMN])
    a_average_d_damage_taken = 0
    if a_death == 0:
        a_average_d_damage_taken = float(a_total_damage_taken)
    else:
        a_average_d_damage_taken = float(a_total_damage_taken) / float(int(a_death))
    AddToDict(player_d_damage_taken_sum_dict, a_player, a_average_d_damage_taken)
    
    if a_idx % 5 == 0:
        temp_total_damage_team = 0
        temp_total_gold_team = 0
        for team_idx in range(5):
            temp_total_damage_team += float(all_region_table[a_idx + team_idx][DAMAGE_TO_CHAMPIONS_COLUMN])
            temp_total_gold_team += float(all_region_table[a_idx + team_idx][TOTAL_GOLD_COLUMN])

    print a_idx
    
    a_gold_share = a_total_gold / temp_total_gold_team
    a_damage_share = float(a_total_damage) / temp_total_damage_team
    a_damage_to_gold = a_total_damage / float(a_total_gold)
    a_damage_to_gold_share = a_damage_share / float(a_gold_share)
    AddToDict(player_damage_to_gold_sum_dict, a_player, a_damage_to_gold)
    AddToDict(player_damage_to_gold_share_sum_dict, a_player, a_damage_to_gold_share)
    AddToDict(player_damage_share_sum_dict, a_player, a_damage_share)
    
    if a_idx % 5 == 0:
        temp_total_damage_taken_team = 0
        for team_idx in range(5):
            temp_total_damage_taken_team += float(all_region_table[a_idx + team_idx][CUSTOM_DAMAGE_TAKEN_COLUMN])
    a_damage_taken_share = a_total_damage_taken / temp_total_damage_taken_team
    AddToDict(player_damage_taken_share_sum_dict, a_player, a_damage_taken_share)
    
    AddToDict(player_kill_sum_dict, a_player, a_kill)
    AddToDict(player_death_sum_dict, a_player, a_death)
    AddToDict(player_assist_sum_dict, a_player, a_assist)
    
    a_win = float(a_row[RESULT_COLUMN])
    AddToDict(player_win_sum_dict, a_player, a_win)
    
    a_champ = a_row[CHAMPION_COLUMN]
    if a_player in player_champs_dict.keys():
        AddToDict(player_champs_dict[a_player], a_champ, 1)
    else:
        player_champs_dict[a_player] = {}
        AddToDict(player_champs_dict[a_player], a_champ, 1)
        
    player_league_dict[a_player] = a_row[LEAGUE_COLUMN]
    player_position_dict[a_player] = a_row[POSITION_COLUMN]
    
    print a_player, a_team, a_dpm, a_gpm, a_average_ka_damage, a_average_d_damage_taken
    if a_idx % 100 == 0:
        print a_idx

405.3279105236
0
ADD MVP 405.327910524 367.778342654 1898.28571429 23349.0
0
273.37061515
1
Beyond MVP 273.37061515 373.81799695 1120.25 8309.66666667
587.615658363
2
Ian MVP 587.615658364 386.171835283 2408.0 5782.0
453.7671581088
3
MaHa MVP 453.767158109 411.855617692 2125.14285714 6576.0
159.2272496187
4
Max MVP 159.227249619 247.68683274 745.714285714 8530.0
397.153024911
5
Expession Longzhu Gaming 397.153024911 320.711743773 4340.0 23223.0
325.6227758007
6
Crash Longzhu Gaming 325.622775801 342.155566853 1525.0 8882.0
532.9537366548
7
Fly Longzhu Gaming 532.953736655 365.033045247 2496.0 8225.5
220.4168784952
8
PraY Longzhu Gaming 220.416878495 326.35485511 7226.0 5680.0
99.5017793594
9
Gorilla Longzhu Gaming 99.5017793595 207.971530249 815.5 4167.33333333
839.3346080306
10
Expession Longzhu Gaming 839.334608031 364.680688337 2438.73333333 4351.0
192.7112810707
11
Crash Longzhu Gaming 192.711281071 357.659655832 839.9 6334.16666667
669.3154875717
12
Fly Longzhu Gaming 669.31548757

In [5]:
#write player data to csv
output_player_def_row = [u"选手",u"每分钟对英雄伤害", u"每分钟金钱", u"平均每击败需要伤害", u"平均每死亡承受伤害",\
                         u"伤害金钱比", u"伤害金钱比（占团队比例）", u"伤害占团队百分比", u"承伤占团队百分比",\
                         u"击杀", u"死亡", u"助攻", u"胜率", u"选取英雄频次", u"赛区", u"位置", u"出场次数"]
with open(output_player_file_name, 'wb') as csvfile:
    csvfile.write(u'\ufeff'.encode('utf8'))
    csvwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow([item.encode('utf8') for item in output_player_def_row])
    
    for a_player in player_num_dict:
        csvwriter.writerow([a_player, 
                            player_dpm_sum_dict[a_player]/float(player_num_dict[a_player]),
                            player_gpm_sum_dict[a_player]/float(player_num_dict[a_player]),
                            player_ka_damage_sum_dict[a_player]/float(player_num_dict[a_player]),
                            player_d_damage_taken_sum_dict[a_player]/float(player_num_dict[a_player]),
                            player_damage_to_gold_sum_dict[a_player]/float(player_num_dict[a_player]),
                            player_damage_to_gold_share_sum_dict[a_player]/float(player_num_dict[a_player]),
                            player_damage_share_sum_dict[a_player]/float(player_num_dict[a_player]),
                            player_damage_taken_share_sum_dict[a_player]/float(player_num_dict[a_player]),
                            player_kill_sum_dict[a_player]/float(player_num_dict[a_player]),
                            player_death_sum_dict[a_player]/float(player_num_dict[a_player]),
                            player_assist_sum_dict[a_player]/float(player_num_dict[a_player]),
                            player_win_sum_dict[a_player]/float(player_num_dict[a_player]),
                            ' '.join('{}-{}'.format(key, val) for key, val in sorted(player_champs_dict[a_player].items())),
                            player_league_dict[a_player],
                            player_position_dict[a_player],
                            player_num_dict[a_player]
                           ])

In [6]:
#统计英雄情况
champ_dpm_sum_dict = {}
champ_gpm_sum_dict = {}
champ_num_dict = {}
champ_ka_damage_sum_dict = {}
champ_d_damage_taken_sum_dict = {}
champ_damage_to_gold_sum_dict = {}
champ_damage_to_gold_share_sum_dict = {}
champ_damage_share_sum_dict = {}
champ_damage_taken_share_sum_dict = {}
champ_kill_sum_dict = {}
champ_death_sum_dict = {}
champ_assist_sum_dict = {}
champ_win_sum_dict = {}
champ_champs_dict = {}
champ_league_dict = {}

champ_ban_sum_dict = {}
champ_ban1_sum_dict = {}
champ_ban2_sum_dict = {}
champ_pick_sum_dict = {}

temp_total_damage_taken_team = 0
temp_total_damage_team = 0
temp_total_gold_team = 0

for a_idx, a_row in enumerate(all_region_table):
    a_player = a_row[PLAYER_COLUMN]
    a_position = a_row[POSITION_COLUMN]
    a_team = a_row[TEAM_COLUMN]
    a_champ = a_row[CHAMPION_COLUMN]
    
    a_champ_position_key = a_champ + '_' + a_position
    
    print a_row[DPM_COLUMN]
    a_total_damage = float(a_row[DAMAGE_TO_CHAMPIONS_COLUMN])
    a_game_length = float(a_row[GAMELENGTH_COLUMN])
    a_dpm = a_total_damage / a_game_length
    a_total_gold = float(a_row[TOTAL_GOLD_COLUMN])
    a_gpm = (a_total_gold - 500) / a_game_length #500 is the starting gold
    
    AddToDict(champ_dpm_sum_dict, a_champ_position_key, a_dpm)
    AddToDict(champ_gpm_sum_dict, a_champ_position_key, a_gpm)
    AddToDict(champ_num_dict, a_champ_position_key, 1)
    
    
    a_kill = int(a_row[KILL_COLUMN])
    a_death = int(a_row[DEATH_COLUMN])
    a_assist = int(a_row[ASSIST_COLUMN])

    a_average_ka_damage = 0
    if a_kill + a_assist == 0:
        a_average_ka_damage = float(a_total_damage)
    else:
        a_average_ka_damage = float(a_total_damage) / float(int(a_kill) + int(a_assist))
    AddToDict(champ_ka_damage_sum_dict, a_champ_position_key, a_average_ka_damage)
    
    a_total_damage_taken = float(a_row[CUSTOM_DAMAGE_TAKEN_COLUMN])
    a_average_d_damage_taken = 0
    if a_death == 0:
        a_average_d_damage_taken = float(a_total_damage_taken)
    else:
        a_average_d_damage_taken = float(a_total_damage_taken) / float(int(a_death))
    AddToDict(champ_d_damage_taken_sum_dict, a_champ_position_key, a_average_d_damage_taken)
    
    if a_idx % 5 == 0:
        temp_total_damage_team = 0
        temp_total_gold_team = 0
        for team_idx in range(5):
            temp_total_damage_team += float(all_region_table[a_idx + team_idx][DAMAGE_TO_CHAMPIONS_COLUMN])
            temp_total_gold_team += float(all_region_table[a_idx + team_idx][TOTAL_GOLD_COLUMN])

    print a_idx
    
    a_gold_share = a_total_gold / temp_total_gold_team
    a_damage_share = float(a_total_damage) / temp_total_damage_team
    a_damage_to_gold = a_total_damage / float(a_total_gold)
    a_damage_to_gold_share = a_damage_share / float(a_gold_share)
    AddToDict(champ_damage_to_gold_sum_dict, a_champ_position_key, a_damage_to_gold)
    AddToDict(champ_damage_to_gold_share_sum_dict, a_champ_position_key, a_damage_to_gold_share)
    AddToDict(champ_damage_share_sum_dict, a_champ_position_key, a_damage_share)
    
    if a_idx % 5 == 0:
        temp_total_damage_taken_team = 0
        for team_idx in range(5):
            temp_total_damage_taken_team += float(all_region_table[a_idx + team_idx][CUSTOM_DAMAGE_TAKEN_COLUMN])
    a_damage_taken_share = a_total_damage_taken / temp_total_damage_taken_team
    AddToDict(champ_damage_taken_share_sum_dict, a_champ_position_key, a_damage_taken_share)
    
    AddToDict(champ_kill_sum_dict, a_champ_position_key, a_kill)
    AddToDict(champ_death_sum_dict, a_champ_position_key, a_death)
    AddToDict(champ_assist_sum_dict, a_champ_position_key, a_assist)
    
    a_win = float(a_row[RESULT_COLUMN])
    AddToDict(champ_win_sum_dict, a_champ_position_key, a_win)
    
    if a_idx % 5 == 0:
        AddToDict(champ_ban_sum_dict, a_row[BAN1_COLUMN], 1)
        AddToDict(champ_ban_sum_dict, a_row[BAN2_COLUMN], 1)
        AddToDict(champ_ban_sum_dict, a_row[BAN3_COLUMN], 1)
        AddToDict(champ_ban_sum_dict, a_row[BAN4_COLUMN], 1)
        AddToDict(champ_ban_sum_dict, a_row[BAN5_COLUMN], 1)
        
        AddToDict(champ_ban1_sum_dict, a_row[BAN1_COLUMN], 1)
        AddToDict(champ_ban1_sum_dict, a_row[BAN2_COLUMN], 1)
        AddToDict(champ_ban1_sum_dict, a_row[BAN3_COLUMN], 1)
        
        AddToDict(champ_ban2_sum_dict, a_row[BAN4_COLUMN], 1)
        AddToDict(champ_ban2_sum_dict, a_row[BAN5_COLUMN], 1)
    if a_idx % 100 == 0:
        print a_idx

405.3279105236
0
0
273.37061515
1
587.615658363
2
453.7671581088
3
159.2272496187
4
397.153024911
5
325.6227758007
6
532.9537366548
7
220.4168784952
8
99.5017793594
9
839.3346080306
10
192.7112810707
11
669.3154875717
12
864.068833652
13
136.3135755258
14
455.3116634799
15
655.5258126195
16
406.0038240918
17
412.1988527725
18
267.5793499044
19
524.4044467972
20
162.2445738486
21
192.0063525675
22
279.2906299629
23
193.4674430916
24
359.936474325
25
363.4621492853
26
417.3001588142
27
350.6934886183
28
197.1201694018
29
367.912568306
30
352.8306010929
31
355.650273224
32
654.1420765027
33
116.5683060109
34
443.825136612
35
266.3169398907
36
497.0054644809
37
461.0491803279
38
402.7759562842
39
292.9717514124
40
359.0960451977
41
628.0677966102
42
315.3446327684
43
415.0282485876
44
536.9265536723
45
146.7570621469
46
346.9830508475
47
391.4576271186
48
408.0903954802
49
308.2890855457
50
340.3539823009
51
533.982300885
52
534.4837758112
53
203.5103244838
54
387.7876106195
55
403.1268436

In [7]:
#write champ data to file
output_champion_def_row = [u"英雄", u"位置", u"每分钟对英雄伤害", u"每分钟金钱", u"平均每击败需要伤害", u"平均每死亡承受伤害",\
                         u"伤害金钱比", u"伤害金钱比（占团队比例）", u"伤害占团队百分比", u"承伤占团队百分比",\
                         u"击杀", u"死亡", u"助攻", u"胜率", u"BAN", u"第一轮ban", u"第二轮ban", u"PICK", u"BAN+PICK"]

match_num = len(all_region_table) / 10
with open(output_champion_file_name, 'wb') as csvfile:
    csvfile.write(u'\ufeff'.encode('utf8'))
    csvwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow([item.encode('utf8') for item in output_champion_def_row])
    
    for a_champ_position in champ_num_dict:
        a_champ = a_champ_position.split('_')[0]
        a_position = a_champ_position.split('_')[1]
        
        ban_rate = 0
        ban1_rate = 0
        ban2_rate = 0
        pick_rate = 0
        if a_champ in champ_ban_sum_dict.keys():
            ban_rate = champ_ban_sum_dict[a_champ]#/float(match_num)
        if a_champ_position in champ_num_dict.keys():
            pick_rate = champ_num_dict[a_champ_position]#/float(match_num)
        if a_champ in champ_ban1_sum_dict.keys():
            ban1_rate = champ_ban1_sum_dict[a_champ]
        if a_champ in champ_ban2_sum_dict.keys():
            ban2_rate = champ_ban2_sum_dict[a_champ]
        csvwriter.writerow([a_champ, 
                            a_position,
                            champ_dpm_sum_dict[a_champ_position]/float(champ_num_dict[a_champ_position]),
                            champ_gpm_sum_dict[a_champ_position]/float(champ_num_dict[a_champ_position]),
                            champ_ka_damage_sum_dict[a_champ_position]/float(champ_num_dict[a_champ_position]),
                            champ_d_damage_taken_sum_dict[a_champ_position]/float(champ_num_dict[a_champ_position]),
                            champ_damage_to_gold_sum_dict[a_champ_position]/float(champ_num_dict[a_champ_position]),
                            champ_damage_to_gold_share_sum_dict[a_champ_position]/float(champ_num_dict[a_champ_position]),
                            champ_damage_share_sum_dict[a_champ_position]/float(champ_num_dict[a_champ_position]),
                            champ_damage_taken_share_sum_dict[a_champ_position]/float(champ_num_dict[a_champ_position]),
                            champ_kill_sum_dict[a_champ_position]/float(champ_num_dict[a_champ_position]),
                            champ_death_sum_dict[a_champ_position]/float(champ_num_dict[a_champ_position]),
                            champ_assist_sum_dict[a_champ_position]/float(champ_num_dict[a_champ_position]),
                            champ_win_sum_dict[a_champ_position]/float(champ_num_dict[a_champ_position]),
                            ban_rate,
                            ban1_rate,
                            ban2_rate,
                            pick_rate,
                            ban_rate + pick_rate,
                           ])